In [10]:
#Task 1: Imports and Utility - We're just doing some imports and enabling async to work within the Jupyter environment here, nothing too crazy!
#!pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [3]:
from utils.text_utils import TextFileLoader, CharacterTextSplitter

In [4]:
from utils.vectordatabase import VectorDatabase
import asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
#Task 2: Documents

# Loading Source Documents

# So, first things first, we need some documents to work with.

# While we could work directly with the .txt files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

# In this case, we're going to parse our text file into a single document in memory.

# Let's look at the relevant bits of the TextFileLoader class:

# def load_file(self):
#         with open(self.path, "r", encoding=self.encoding) as f:
#             self.documents.append(f.read())

# We're simply loading the document using the built in open method, 
#and storing that output in our self.documents list.


In [6]:
text_loader = TextFileLoader("data/kingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [7]:
print(documents[0][:100])

1606
THE TRAGEDY OF KING LEAR

by William Shakespeare




Dramatis Personae

      Lear, King of Bri


In [12]:
#Splitting Text Into Chunks

# As we can see, there is one massive document.

# We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

# There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data very well in order to build the most robust system.

# For this toy example, we'll just split blindly on length.

#     There's an opportunity to clear up some terminology here, for this course we will be stick to the following:

#         "source documents" : The .txt, .pdf, .html, ..., files that make up the files and information we start with in its raw format
#         "document(s)" : single (or more) text object(s)
#         "corpus" : the combination of all of our documents



In [8]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

216

In [13]:
#Let's take a look at some of the documents we've managed to split.

In [9]:
split_documents[0:1]

["1606\nTHE TRAGEDY OF KING LEAR\n\nby William Shakespeare\n\n\n\n\nDramatis Personae\n\n      Lear, King of Britain.\n      King of France.\n      Duke of Burgundy.\n      Duke of Cornwall.\n      Duke of Albany.\n      Earl of Kent.\n      Earl of Gloucester.\n      Edgar, son of Gloucester.\n      Edmund, bastard son to Gloucester.\n      Curan, a courtier.\n      Old Man, tenant to Gloucester.\n      Doctor.\n      Lear's Fool.\n      Oswald, steward to Goneril.\n      A Captain under Edmund's command.\n      Gentlemen.\n      A Herald.\n      Servants to Cornwall.\n\n      Goneril, daughter to Lear.\n      Regan, daughter to Lear.\n      Cordelia, daughter to Lear.\n\n      Knights attending on Lear, Officers, Messengers, Soldiers,\n        Attendants.\n\n\n\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF CARNEGIE MELLON UNIVERSITY\nWITH PERMISSION.  ELECTRONIC 

In [14]:

# Task 3: Embeddings and Vectors

# Next, we have to convert our corpus into a "machine readable" format 

# we're going to see the actual process of creating, and then storing, 
# these embeddings, and how we can leverage that to intelligently add context to our queries.
# OpenAI API Key

# In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!


In [17]:
import os
import openai
from openai import OpenAI
%load_ext dotenv
%dotenv

In [18]:

# Vector Database

# Let's set up our vector database to hold all our documents and their embeddings!

# While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

# Let's look at our VectorDatabase().__init__():

# def __init__(self, embedding_model: EmbeddingModel = None):
#         self.vectors = defaultdict(np.array)
#         self.embedding_model = embedding_model or EmbeddingModel()

# As you can see - our vectors are merely stored as a dictionary of np.array objects.

# Secondly, our VectorDatabase() has a default EmbeddingModel() which is a wrapper for OpenAI's text-embedding-3-small model.

#     Quick Info About text-embedding-3-small:

#         It has a context window of 8191 tokens
#         It returns vectors with dimension 1536



In [19]:


# We can call the async_get_embeddings method of our EmbeddingModel() on a list of str and receive a list of float back!

# async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
#         return await aget_embeddings(
#             list_of_text=list_of_text, engine=self.embeddings_model_name
#         )

# We cast those to np.array when we build our VectorDatabase():

# async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
#         embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
#         for text, embedding in zip(list_of_text, embeddings):
#             self.insert(text, np.array(embedding))
#         return self

# And that's all we need to do!


In [20]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [21]:


# So, to review what we've done so far in natural language:

#     We load source documents
#     We split those source documents into smaller chunks (documents)
#     We send each of those documents to the text-embedding-3-small OpenAI API endpoint
#     We store each of the text representations with the vector representations as keys/values in a dictionary

# Semantic Similarity

# The next step is to be able to query our VectorDatabase() with a str and have it return to us vectors and text that is most relevant from our corpus.

# We're going to use the following process to achieve this in our toy example:

#     We need to embed our query with the same EmbeddingModel() as we used to construct our VectorDatabase()
#     We loop through every vector in our VectorDatabase() and use a distance measure to compare how related they are
#     We return a list of the top k closest vectors, with their text representations

# There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

#     We are using cosine similarity as a distance metric in this example - but there are many many distance metrics you could use - like these

#     We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like ANN



In [22]:
vector_db.search_by_text("What is the climax on the KingLear story?", k=3)

[("and mortified bare arms\n     Pins, wooden pricks, nails, sprigs of rosemary;\n     And with this horrible object, from low farms, \n     Poor pelting villages, sheepcotes, and mills,\n     Sometime with lunatic bans, sometime with prayers,\n     Enforce their charity. 'Poor Turlygod! poor Tom!'\n     That's something yet! Edgar I nothing am.             Exit.\n\n\n\n\nScene IV.\nBefore Gloucester's Castle; Kent in the stocks.\n\nEnter Lear, Fool, and Gentleman.\n\n  Lear. 'Tis strange that they should so depart from home,\n     And not send back my messenger.\n  Gent. As I learn'd,\n     The night before there was no purpose in them\n     Of this remove.\n  Kent. Hail to thee, noble master!\n  Lear. Ha!\n     Mak'st thou this shame thy pastime?\n  Kent. No, my lord.\n  Fool. Ha, ha! look! he wears cruel garters. Horses are tied by\nthe\n     head, dogs and bears by th' neck, monkeys by th' loins, and\nmen\n     by th' legs. When a man's over-lusty at legs, then he wears\n     woode